In [31]:
print(__doc__)

# Code source adapted from: Jaques Grobler
# License: BSD 3 clause

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
import pandas
import os
import csv

Automatically created module for IPython interactive environment


In [51]:
def treat_dataset(dataset):
    vcut = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}
    for k,c in dataset["cut"].items():
        dataset["cut"][k] = vcut[c]
    print(dataset)

In [ ]:
dataset_train = pandas.read_csv('diamonds-train.csv')
treat_dataset(dataset_train)
print(dataset_train)

/home/matheusmortatti/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
